# Backpropagation through Back substitution with a Backslash
Notebook per il seminario d'esame per il corso di Metodi di Approssimazione 

## Algebra degli operatori

Definiamo il tipo di dato custom che rappresenta la funzione lineare

In [1]:
using LinearAlgebra

struct Operator  # Linear Matrix Operators from Matrices to Matrices (and the operator adjoint)
    op
    adj
    sym
end

## Operators
ℒ(A::Matrix)  = Operator(X->A*X   , X->A'*X, "ℒ$(size(A))"  )   # left multiply by A (X → AX)
ℛ(A::Matrix)  = Operator(X->X*A   , X->X*A', "ℛ$(size(A))")     # right multiply by A (X → XA)
ℋ(A::Matrix)  = Operator(X->X.*A  , X->X.*A, "ℋ$(size(A))")    # Hadamard product (elementwise product)
ℐ()  =          Operator(X->X      ,    X->X,    "I")     # identity operator
𝒪()  =           Operator(X->zero(X) , X->zero(X),"𝒪")# zero operator

𝒪 (generic function with 1 method)

Dobbiamo anche fare *overloading* delle operazioni

In [2]:
import Base:  zero, show, adjoint, *, \, ∘, +, -
show(io::IO, M::Operator) = print(io, M.sym)  # pretty printing
zero(::Any) = 𝒪() # Let's make any undefined zero the 𝒪 operator
adjoint(A::Operator) = Operator(A.adj, A.op,  "("*A.sym*")'")
adjoint(B::Bidiagonal) = Bidiagonal(adjoint.(B.dv),adjoint.(B.ev),(B.uplo == 'U') ? :L : :U) # lower to upper

-(A::Operator) = Operator(X->-A.op(X), X->-A.adj(X),"-"*A.sym)
-(::typeof(𝒪), X::Matrix) = -X # 𝒪 - X should be -X
*(A::Operator, X::Matrix) = A.op(X)
\(ℐ::typeof(ℐ()), A::Matrix) = A
∘(A::Operator, B::Operator) = Operator(A.op ∘ B.op, B.adj ∘ A.adj, A.sym*"∘"*B.sym)
# We need [A;B]*C to somehow magically be [AC;BC]
*(M::Adjoint{Operator, Matrix{Operator}},v::Array) = M .* [v]
+(A::Array,x::Number)=A.+x

+ (generic function with 192 methods)

## Esempi

Vediamo qualche esempio.

In [4]:
# Basic Test
B = [ 1 2; 3 4]
M = [10 1;1 10]
C = [ 2 5;4 6]

LM = ℒ(M)

ℒ(2, 2)

### `Operator * Matrix`

A parte la negazione `-`, la prima regola che abbiamo scritto è
```
*(A::Operator, X::Matrix) = A.op(X)
```
Ossia, il prodotto tra un operatore e una matrice consiste nell'applicare l'operatore alla matrice. 

Iniziamo guardando la moltiplicazione a sinistra

In [9]:
typeof(LM), typeof(I(2))

(Operator, Diagonal{Bool, Vector{Bool}})

In [6]:
LM * [1 0; 0 1]

2×2 Matrix{Int64}:
 10   1
  1  10

Osserviamo che abbiamo definito il prodotto *a destra* per una matrice. Se moltiplichiamo a sinistra per una matrice, le cose non tornano 

In [53]:
try
    [1 0; 0 1] * LM
catch e
    print("Non si può fare questa moltiplicazione. Ecco l'errore che dà:\n\n")
    showerror(stdout, e)
end

Non si può fare questa moltiplicazione. Ecco l'errore che dà:

MethodError: no method matching *(::Matrix{Int64}, ::Operator)
The function `*` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  *(::Any, ::Any, ::Any, ::Any...)
   @ Base operators.jl:642
  *(::AbstractMatrix, ::LinearAlgebra.AbstractQ)
   @ LinearAlgebra ~/.julia/juliaup/julia-1.12.4+0.x64.linux.gnu/share/julia/stdlib/v1.12/LinearAlgebra/src/abstractq.jl:201
  *(::AbstractMatrix, ::LinearAlgebra.AbstractRotation)
   @ LinearAlgebra ~/.julia/juliaup/julia-1.12.4+0.x64.linux.gnu/share/julia/stdlib/v1.12/LinearAlgebra/src/givens.jl:20
  ...


... e ci sono altri piccoli problemini per il fatto che abbiamo usato `Matrix` 

In [54]:
try
    LM .* I(2)
catch e
    print("No, non si può fare:\n\n")
    showerror(stdout, e)
end

No, non si può fare:

MethodError: no method matching length(::Operator)
The function `length` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  length(::Compiler.MethodLookupResult)
   @ Base ~/.julia/juliaup/julia-1.12.4+0.x64.linux.gnu/share/julia/Compiler/src/methodtable.jl:10
  length(::Compiler.NewNodeStream)
   @ Base ../usr/share/julia/Compiler/src/ssair/ir.jl:366
  length(::Profile.HeapSnapshot.Edges)
   @ Profile ~/.julia/juliaup/julia-1.12.4+0.x64.linux.gnu/share/julia/stdlib/v1.12/Profile/src/heapsnapshot_reassemble.jl:24
  ...


Ma torniamo alle cose che funzionano

In [17]:
LM * B

2×2 Matrix{Int64}:
 13  24
 31  42

E in effetti è la stessa cosa che fare...

In [18]:
LM.op(B)

2×2 Matrix{Int64}:
 13  24
 31  42

Vediamo ora la moltiplicazione a destra

In [22]:
ℛ(M) * B # i.e. ℛ(M).op(B) 

2×2 Matrix{Int64}:
 12  21
 34  43

In [20]:
B * M # right multiply by M

2×2 Matrix{Int64}:
 12  21
 34  43

La notazione appare controintuitiva, ma ha senso in realtà: `ℛ(M) * B ` denota l'applicazione dell'operatore `ℛ(M)` a `B`, ovvero `ℛ(M).op(B)`.

Per completezza, vediamo il prodotto di Hadamard

In [23]:
[ℋ(M) * B M .* B]

2×4 Matrix{Int64}:
 10   2  10   2
  3  40   3  40

Ricordiamo che in Julia `.*` vuol dire moltiplicazione *pointwise*

### Trasposizione di operatori

Tra le varie cose, abbiamo scritto
```
adjoint(A::Operator) = Operator(A.adj, A.op,  "("*A.sym*")'")
```
Ovvero abbiamo definito la trasposizione tra operatori. Infatti ricordiamo che in Julia `A'` essenzialmente viene tradotto in `adjoint(A)`.

Ricordiamo inoltre **finire: ' è involutorio e ricorsivo**

Verifichiamo che la trasposizione di operatori faccia il suo dovere, ossia che 
$$\langle B, \mathcal{L}\,C\rangle=\langle\mathcal{L}'\,B, C\rangle$$
dove $\mathcal{L}'$ denota l'operatore aggiunto $\mathcal{L}^{\top}$

In [43]:
print("<B,ℒC> = $(tr( B'*(ℒ(M)*C) )),\t <ℒ'B,C> = $(tr( (ℒ(M)'*B)' * C ))")   # <ℒ'B,C>

<B,ℒC> = 522,	 <ℒ'B,C> = 522

In termini un po' più "espliciti"

In [44]:
tr(B' * ℒ(M).op(C)), tr( ℒ(M).adj(B)' * C)

(522, 522)

### `Operator ∘ Operator`

Un'altra importante regola che abbiamo scritto è
```
∘(A::Operator, B::Operator) = Operator(A.op ∘ B.op, B.adj ∘ A.adj, A.sym*"∘"*B.sym)
```
Che definisce la composizione tra operatori

La composizione fa da prodotto nell'algebra degli endomorfismi di uno spazio vettoriale. 

Quindi potrebbe essere ragionevole denotare la composizione con `*`

In [55]:
try
    comp = ℒ(M) * ℒ(M)
catch e
    print("Errore!\n\n")
    showerror(stdout, e)
    println()
end

Errore!

MethodError: no method matching *(::Operator, ::Operator)
The function `*` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  *(::Any, ::Any, ::Any, ::Any...)
   @ Base operators.jl:642
  *(::Operator, ::Matrix)
   @ Main ~/Scrivania/MA Lab/MA-project/Codice/test/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W5sZmlsZQ==.jl:9
  *(::BitMatrix, ::UniformScaling)
   @ LinearAlgebra ~/.julia/juliaup/julia-1.12.4+0.x64.linux.gnu/share/julia/stdlib/v1.12/LinearAlgebra/src/uniformscaling.jl:264
  ...



In [ ]:
comp1 = ℒ(M) ∘ ℒ(M)

ℒ(2, 2)∘ℒ(2, 2)

In [ ]:
comp1 * [1 0; 0 1]

2×2 Matrix{Int64}:
 101   20
  20  101

fa ciò che ci si aspetta:

In [56]:
(M * M) * [1 0; 0 1]

2×2 Matrix{Int64}:
 101   20
  20  101

Vediamo un altro esempio

In [57]:
comp2 = ℒ(M) ∘ ℛ(C)

ℒ(2, 2)∘ℛ(2, 2)

In [61]:
comp2 * B       # (ℒ(M) ∘ ℛ(C))[B]

2×2 Matrix{Int64}:
 122  209
 230  407

Sarebbe

In [59]:
ℒ(M).op(ℛ(C).op(B))

2×2 Matrix{Int64}:
 122  209
 230  407

ovvero

In [60]:
M * B * C

2×2 Matrix{Int64}:
 122  209
 230  407